# Gross inland consumption of bioenergy in the EU27 (Czech output tables)

Based on Eurostat [energy balances](https://ec.europa.eu/eurostat/web/energy/data/energy-balances) data from 2021. 

Methology of energy balances is different from the [SHARES](https://ec.europa.eu/eurostat/web/energy/data/shares) database which is used to identify share of renewables on final energy consumption.

In [1]:
import pandas as pd
import os

In [2]:
# Get iso codes of EU member states from wikipedia
eu_df = pd.read_html('https://en.wikipedia.org/wiki/Member_state_of_the_European_Union')[1]
member = eu_df.loc[:26, 'ISO 3166-1 alpha-2']

In [3]:
# Get iso codes of all countries from wikipedia
iso_df = pd.read_html('https://cs.wikipedia.org/wiki/ISO_3166-1')[0]

In [4]:
# Rename default columns for easier merging
iso_df = iso_df[['alpha-2', 'Stát']]
iso_df = iso_df.rename(columns={'alpha-2': 'alpha_2', 'Stát': 'country'})

In [5]:
# Load Eurostat balances csv for bioenergy
csvfile = os.path.abspath('balances_bioenergy_2002_2019_tj.csv')
df0 = pd.read_csv(csvfile, delimiter=',', decimal=',')

In [6]:
# Change country code for Greece (Eurostat uses 'EL' to encode the file)
df0.loc[df0['country'] == 'EL', 'country'] = 'GR'

In [7]:
# Merge to have human readable country names
df0.rename(columns={'country': 'alpha_2'}, inplace=True)
df0 = df0[df0['alpha_2'].isin(member)]
df = df0.merge(iso_df, on='alpha_2', how='left')

In [8]:
df.head()

,alpha_2,year,fuel,exports,gross_inland_consumption,imports,primary_production,dependency,country
0,AT,2002,bioenergy,8517.123504,121985.976780,8231.248800,122271.893352,-0.002344,Rakousko
1,AT,2002,biofuels,0.000000,851.888196,0.000000,851.888196,0.000000,Rakousko
2,AT,2002,biogas,0.000000,919.002600,0.000000,919.002600,0.000000,Rakousko
3,AT,2002,ren_mun_waste,0.000000,1970.015004,0.000000,1970.015004,0.000000,Rakousko
4,AT,2002,renewables,7233.994908,271396.498392,6886.992924,271743.500376,-0.001279,Rakousko


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3402 entries, 0 to 3401
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   alpha_2                   3402 non-null   object 
 1   year                      3402 non-null   int64  
 2   fuel                      3402 non-null   object 
 3   exports                   3402 non-null   float64
 4   gross_inland_consumption  3402 non-null   float64
 5   imports                   3402 non-null   float64
 6   primary_production        3402 non-null   float64
 7   dependency                3150 non-null   float64
 8   country                   3402 non-null   object 
dtypes: float64(5), int64(1), object(3)
memory usage: 265.8+ KB


In [10]:
bioenergy = df['fuel'] == 'bioenergy'
total = df['fuel'] == 'total'
res = df['fuel'] == 'renewables'

## Gross inland consumption of bioenergy

In [11]:
consumption_bioenergy = df.loc[
    bioenergy,
    ['year', 'country', 'gross_inland_consumption']
    ].pivot(
        index='year',
        columns='country'
    ).droplevel(0, axis=1)
    
consumption_bioenergy.index.name = 'Rok'
consumption_bioenergy.columns.name = None
consumption_bioenergy.head()

,Belgie,Bulharsko,Chorvatsko,Dánsko,Estonsko,Finsko,Francie,Irsko,Itálie,Kypr,...,Polsko,Portugalsko,Rakousko,Rumunsko,Slovensko,Slovinsko,Česko,Řecko,Španělsko,Švédsko
Rok,,,,,,,,,,,,,,,,,,,,,
2002,28346.980608,26769.017556,43865.019864,69169.201704,22918.291992,292090.469148,400732.528176,6242.602536,132742.284660,516.986064,...,164661.610500,115218.391392,121985.976780,98477.429724,11786.511888,22816.636488,72605.182860,41723.052984,176862.950532,308707.920216
2003,35113.686768,28538.987256,50698.003068,78701.707944,24086.827872,295242.501528,424164.875472,6058.550808,203210.901612,604.992600,...,166789.635336,115339.641120,126972.078768,119058.608088,14156.952444,25428.236724,71296.179840,39403.523916,193688.820504,325832.937048
2004,38901.275388,29621.986812,49787.080992,86527.339560,24945.205608,310402.443780,432107.193204,7187.144616,180444.883536,556.802532,...,172360.172736,117275.575572,128553.433128,131216.698476,15423.082632,25163.463492,76443.013080,39903.260364,197991.594864,332264.824812
2005,46704.926304,28980.987732,52238.871072,94692.562524,24355.452960,296555.021460,457780.567068,9072.669996,253224.655956,516.399912,...,178559.986176,119244.543876,154127.956644,133346.942316,16783.122744,29706.183360,78234.461064,41472.556740,206199.816264,343611.555228
2006,55181.437848,31053.286260,48550.174668,96245.321040,21936.822336,327295.595628,421888.470444,9274.724964,290972.886624,521.382204,...,187260.282180,122623.542684,179539.069356,133336.893996,18589.643208,26416.656468,82671.003684,42323.398236,202487.506308,368384.306544


In [12]:
consumption_bioenergy.to_csv('domaci_spotreba_eu_tj.csv')

In [13]:
consumption = df.loc[:, ['country', 'year', 'fuel', 'gross_inland_consumption']].pivot(index=['country', 'year'], columns='fuel').droplevel(0, axis=1).reset_index()

## Transformations to get shares

In [14]:
consumption.columns.name = None
consumption['share_bioenergy_total'] = consumption['bioenergy'] / consumption['total']
consumption['share_bioenergy_renewables'] = consumption['bioenergy'] / consumption['renewables']
consumption['share_renewables_total'] = consumption['renewables'] / consumption['total']
consumption.head()

,country,year,bioenergy,biofuels,biogas,ren_mun_waste,renewables,solid_biomass,total,share_bioenergy_total,share_bioenergy_renewables,share_renewables_total
0,Belgie,2002,28346.980608,0.000000,1909.013328,5663.986776,29793.185064,20533.993128,2.377972e+06,0.011921,0.951459,0.012529
1,Belgie,2003,35113.686768,0.000000,2156.997492,7620.017868,36311.990796,25103.006100,2.484207e+06,0.014135,0.967000,0.014617
2,Belgie,2004,38901.275388,0.000000,3025.004868,7201.002924,40434.439680,28336.011192,2.491045e+06,0.015616,0.962083,0.016232
3,Belgie,2005,46704.926304,718.036200,3886.983252,7840.997172,48390.866928,33842.992968,2.477052e+06,0.018855,0.965160,0.019536
4,Belgie,2006,55181.437848,2432.991348,3340.982664,7723.013148,57136.003560,40730.990724,2.449480e+06,0.022528,0.965791,0.023326


In [15]:
consumption[['country', 'year', 'share_bioenergy_total', 'share_bioenergy_renewables', 'share_renewables_total']].head()

,country,year,share_bioenergy_total,share_bioenergy_renewables,share_renewables_total
0,Belgie,2002,0.011921,0.951459,0.012529
1,Belgie,2003,0.014135,0.967000,0.014617
2,Belgie,2004,0.015616,0.962083,0.016232
3,Belgie,2005,0.018855,0.965160,0.019536
4,Belgie,2006,0.022528,0.965791,0.023326


## Share of bioenergy on total gross inland consumption

In [16]:
share_bioenergy_total = consumption[['country', 'year', 'share_bioenergy_total']].pivot(index='year', columns='country').droplevel(0, axis=1)
share_bioenergy_total.index.name = 'Rok'
share_bioenergy_total.columns.name = None
share_bioenergy_total.to_csv('domaci_spotreba_podil_bioenergie_celek.csv')
share_bioenergy_total.tail()

,Belgie,Bulharsko,Chorvatsko,Dánsko,Estonsko,Finsko,Francie,Irsko,Itálie,Kypr,...,Polsko,Portugalsko,Rakousko,Rumunsko,Slovensko,Slovinsko,Česko,Řecko,Španělsko,Švédsko
Rok,,,,,,,,,,,,,,,,,,,,,
2015,0.055959,0.064843,0.156554,0.208182,0.158718,0.278273,0.056430,0.030166,0.088092,0.024602,...,0.081420,0.122884,0.176236,0.117062,0.073765,0.109165,0.090394,0.054716,0.055193,0.234502
2016,0.055098,0.071315,0.153342,0.215620,0.155307,0.274516,0.061648,0.033630,0.086804,0.024176,...,0.073668,0.121626,0.172789,0.122016,0.070771,0.103984,0.092164,0.049734,0.056061,0.233755
2017,0.055799,0.069153,0.150061,0.233234,0.171782,0.288335,0.061757,0.039900,0.087434,0.028735,...,0.069166,0.117109,0.170400,0.117926,0.068258,0.098473,0.089991,0.048707,0.056153,0.244346
2018,0.058355,0.089229,0.159569,0.233068,0.174775,0.291342,0.062349,0.040736,0.086969,0.031332,...,0.071266,0.120878,0.165480,0.112553,0.071876,0.096605,0.089904,0.049230,0.059069,0.228121
2019,0.052816,0.095245,0.166324,0.247394,0.237399,0.302785,0.064178,0.041582,0.089371,0.031570,...,0.077589,0.127720,0.158609,0.117750,0.101621,0.098867,0.097571,0.046300,0.060942,0.242913


## Share of bioenergy on renewables gross inland consumption

In [17]:
share_bioenergy_res = consumption[['country', 'year', 'share_bioenergy_renewables']].pivot(index='year', columns='country').droplevel(0, axis=1)
share_bioenergy_res.index.name = 'Rok'
share_bioenergy_res.columns.name = None
share_bioenergy_res.to_csv('domaci_spotreba_podil_bioenergie_oze.csv')
share_bioenergy_res.tail()

,Belgie,Bulharsko,Chorvatsko,Dánsko,Estonsko,Finsko,Francie,Irsko,Itálie,Kypr,...,Polsko,Portugalsko,Rakousko,Rumunsko,Slovensko,Slovinsko,Česko,Řecko,Španělsko,Švédsko
Rok,,,,,,,,,,,,,,,,,,,,,
2015,0.811483,0.583396,0.674162,0.715265,0.924847,0.866246,0.602185,0.389248,0.522240,0.362809,...,0.857470,0.520740,0.591563,0.624651,0.761381,0.637605,0.866905,0.464020,0.397194,0.538647
2016,0.793155,0.645072,0.651943,0.734218,0.931499,0.873592,0.598398,0.442365,0.514721,0.367593,...,0.831631,0.459413,0.571291,0.626449,0.733610,0.597672,0.865978,0.435054,0.386893,0.579368
2017,0.773460,0.670760,0.697714,0.722483,0.920786,0.837285,0.607823,0.438218,0.483912,0.413851,...,0.804437,0.529126,0.571086,0.654525,0.739565,0.618033,0.861533,0.407555,0.429044,0.582217
2018,0.760652,0.667184,0.634031,0.726615,0.934358,0.842541,0.567548,0.414201,0.466273,0.346162,...,0.828977,0.475382,0.562430,0.626223,0.774784,0.573890,0.857287,0.373402,0.412685,0.576284
2019,0.685653,0.726315,0.684382,0.710811,0.971303,0.848151,0.566800,0.382955,0.470695,0.337153,...,0.814463,0.504015,0.532655,0.648369,0.785119,0.582039,0.853868,0.373785,0.409675,0.567475


## Share of renewables on total gross inland consuption

In [18]:
share_res_total = consumption[['country', 'year', 'share_renewables_total']].pivot(index='year', columns='country').droplevel(0, axis=1)
share_res_total.index.name = 'Rok'
share_res_total.columns.name = None
share_res_total.to_csv('domaci_spotreba_podil_oze_celek.csv')
share_res_total.tail()

,Belgie,Bulharsko,Chorvatsko,Dánsko,Estonsko,Finsko,Francie,Irsko,Itálie,Kypr,...,Polsko,Portugalsko,Rakousko,Rumunsko,Slovensko,Slovinsko,Česko,Řecko,Španělsko,Švédsko
Rok,,,,,,,,,,,,,,,,,,,,,
2015,0.068959,0.111148,0.232221,0.291056,0.171616,0.321240,0.093709,0.077499,0.168681,0.067809,...,0.094954,0.235979,0.297915,0.187404,0.096883,0.171211,0.104272,0.117917,0.138958,0.435355
2016,0.069466,0.110554,0.235207,0.293672,0.166728,0.314238,0.103022,0.076023,0.168644,0.065767,...,0.088583,0.264741,0.302454,0.194774,0.096469,0.173981,0.106428,0.114316,0.144900,0.403466
2017,0.072142,0.103096,0.215075,0.322822,0.186560,0.344370,0.101604,0.091051,0.180682,0.069434,...,0.085981,0.221325,0.298379,0.180170,0.092295,0.159332,0.104454,0.119511,0.130879,0.419683
2018,0.076717,0.133740,0.251674,0.320758,0.187054,0.345790,0.109857,0.098349,0.186520,0.090511,...,0.085969,0.254275,0.294223,0.179732,0.092769,0.168333,0.104870,0.131842,0.143133,0.395848
2019,0.077031,0.131135,0.243028,0.348045,0.244412,0.356994,0.113229,0.108582,0.189870,0.093636,...,0.095264,0.253406,0.297771,0.181610,0.129434,0.169864,0.114269,0.123869,0.148756,0.428060


## Bioenergy share in context
Share of bioenergy, other renewables and non-renewables on gross inland consumption in 2019 in EU27.

In [19]:
consumption['renewables_without_bioenergy'] =  consumption['renewables'] - consumption['bioenergy']

In [20]:
consumption['share_renewables_without_bioenergy_total'] = consumption['renewables_without_bioenergy'] / consumption['total']

In [21]:
consumption['share_nonrenewables_total'] = (consumption['total'] - consumption['renewables']) / consumption['total']

In [22]:
shares_2019 = consumption.loc[consumption['year'] == 2019, ['country', 'share_bioenergy_total', 'share_renewables_without_bioenergy_total', 'share_nonrenewables_total']]
shares_2019.sort_values(by='share_bioenergy_total', ascending=False)

,country,share_bioenergy_total,share_renewables_without_bioenergy_total,share_nonrenewables_total
215,Lotyšsko,0.352823,0.039147,0.608030
107,Finsko,0.302785,0.054209,0.643006
71,Dánsko,0.247394,0.100651,0.651955
485,Švédsko,0.242913,0.185147,0.571940
89,Estonsko,0.237399,0.007014,0.755588
197,Litva,0.193641,0.009984,0.796375
53,Chorvatsko,0.166324,0.076704,0.756972
359,Rakousko,0.158609,0.139162,0.702229
341,Portugalsko,0.127720,0.125685,0.746594
377,Rumunsko,0.117750,0.063860,0.818390


In [23]:
shares_2019.sum(axis=1)

17     1.0
35     1.0
53     1.0
71     1.0
89     1.0
107    1.0
125    1.0
143    1.0
161    1.0
179    1.0
197    1.0
215    1.0
233    1.0
251    1.0
269    1.0
287    1.0
305    1.0
323    1.0
341    1.0
359    1.0
377    1.0
395    1.0
413    1.0
431    1.0
449    1.0
467    1.0
485    1.0
dtype: float64

In [24]:
shares_2019.sort_values(by='share_bioenergy_total', ascending=False).to_csv('podily_2019.csv', index=False)